In [5]:
# https://www.gettingstarted.ai
# email: jeff@gettingstarted.ai
# written by jeff

# Don't forget to set your OPEN_AI_KEY
# In your terminal execute this command: export OPENAI_API_KEY="YOUR_KEY_HERE"

# Import required modules from the LangChain package
import os
from dotenv import load_dotenv

from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings

load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Load a PDF document and split it into sections
loader = TextLoader("docs/시간의 흔적.txt")
docs = loader.load_and_split()

# Initialize the OpenAI chat model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.8)

# Initialize the OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Load the Chroma database from disk
chroma_db = Chroma(persist_directory="data", 
                   embedding_function=embeddings,
                   collection_name="lc_chroma_demo")

# Get the collection from the Chroma database
collection = chroma_db.get()

# If the collection is empty, create a new one
if len(collection['ids']) == 0:
    # Create a new Chroma database from the documents
    chroma_db = Chroma.from_documents(
        documents=docs, 
        embedding=embeddings, 
        persist_directory="data",
        collection_name="lc_chroma_demo"
    )

    # Save the Chroma database to disk
    chroma_db.persist()

# Prepare query
query = "이 소설의 논점"

print('Similarity search:')
print(chroma_db.similarity_search(query))

print('Similarity search with score:')
print(chroma_db.similarity_search_with_score(query))

# Add a custom metadata tag to the first document
docs[0].metadata = {
    "tag": "demo",
}

# Update the document in the collection if the collection is not empty
if collection['ids']:
    chroma_db.update_document(
        document=docs[0],
        document_id=collection['ids'][0]
    )

    # Find the document with the custom metadata tag
    collection = chroma_db.get(where={"tag" : "demo"})
else:
    print("The collection is empty. Cannot update the document.")

# Prompt the model
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=chroma_db.as_retriever())

# Execute the chain
response = chain(query)

# Print the response
print(response['result'])

# Delete the collection
chroma_db.delete_collection()

Similarity search:
[Document(metadata={'source': 'docs/시간의 흔적.txt'}, page_content='처음에는 모든 것이 낯설었다. 새로운 직장 동료들과의 관계를 쌓는 것이 쉽지 않았고, 회사의 문화에 적응하는 데에도 시간이 걸렸다. 그는 그동안 배운 것들이 과연 이 새로운 환경에서 통할 수 있을지, 혹은 이제까지의 선택이 잘못된 것은 아닌지에 대한 의문이 들기도 했다. 하지만 그는 그 불확실성을 두려워하지 않기로 했다.\n\n"불확실성은 성장의 일부야." 그는 자신에게 그렇게 말하며, 매일 새로운 도전에 맞서기로 다짐했다. 그는 불확실한 미래를 두려워하기보다는 그 속에서 새로운 기회를 찾아가기로 했다.\n\n그가 겪고 있는 변화와 도전은 그의 인생에서 중요한 전환점이었다. 그는 이제 단순히 \'목표를 향해 나아가는 것\'이 아니라, \'어떤 과정과 사람들과 함께 성장할 것인가\'에 대해 고민하기 시작했다. 그는 지금까지의 삶에서 중요한 가치를 찾아왔지만, 그 가치를 어떻게 지속적으로 이어갈 수 있을지에 대한 새로운 질문이 그를 압박하고 있었다.\n\n26장. 새로운 사람들, 새로운 관계\n\n윤석훈이 새로운 도시에서의 삶을 적응해가면서, 그는 점차 사람들과의 관계에서 변화가 일어나는 것을 느꼈다. 직장에서의 동료들과는 점차 친해졌고, 친구들도 하나둘씩 생기기 시작했다. 그 중에서도 특히 민준이라는 동료와 친해졌다. 민준은 이전 직장에서의 경험이 풍부한 사람으로, 윤석훈에게 많은 것을 가르쳐줄 수 있었다.\n\n어느 날, 민준은 그에게 다가와 말했다. “석훈 씨, 우리가 진정으로 성장하는 순간은 우리가 가진 능력이나 지식이 아니라, 우리가 사람들과 어떻게 연결되느냐에 달려있는 것 같아요. 결국 사람들과의 관계가 우리의 삶을 의미 있게 만든다고 생각해요.”\n\n윤석훈은 민준의 말에 깊이 공감했다. 그는 이제 사람들과의 관계 속에서 진정한 의미를 찾고 있다는 것을 느꼈다. 그동안 \'무한한 가능성\'을 찾기 위해 혼